# **Trabajo Práctico Final NLP**



## **Ejercicio 1 - RAG**
**Crear un chatbot experto en un tema a elección, usando la técnica RAG. Como fuentes de conocimiento se utilizarán al menos las siguientes fuentes:**

* **Documentos de texto**
*  **Datos numéricos en formato tabular (por ej., Dataframes, CSV, sqlite, etc.)**
*  **Base de datos de grafos (Online o local)**


`Elijo que el chatbot se especialice en historia de la Segunda Guerra Mundial`

### **Set de datos**

**`Documentos de textos`**

[**Segunda Guerra Mundial - Wikipedia**](https://es.wikipedia.org/wiki/Segunda_Guerra_Mundial)

[**Breve Historia de la Segunda Guerra Mundial - Jesús Hernández**](https://sgfm.elcorteingles.es/SGFM/dctm/MEDIA02/CONTENIDOS/201503/17/00106518013690____4_.pdf)

[**LA SEGUNDA GUERRA MUNDIAL (1939-1945) - profesoremilio**](https://profesoremilio.files.wordpress.com/2018/09/tema-8-la-segunda-guerra-mundial.pdf)

[**LA SEGUNDA GUERRA MUNDIAL (1939-1945) - IES Fray Pedro de Urbina – Departamento de Geografía e Historia**](https://www.iesfraypedro.com/files/sociales/segunda-guerra-mundial-1b.pdf)

[**La II Guerra Mundial - Sabuco**](https://www.sabuco.com/historia/IIGMb.pdf)

**`Datos tabulares`**

[**WW2 - Kaggle**](https://www.kaggle.com/datasets/ramjasmaurya/world-war-2-archive?select=events.csv)


---

Nota: de los archivos tabulares de Kaggle solo utilizo el que corresponde a los eventos. El mismo fue traducido al español para mejorar la compresión de las solicitudes.

### **Librerías**

In [ ]:
!pip install wikipedia-api
!pip install pdfminer.six
!pip install gdown
!pip install llama_index==0.10.0 sentence-transformers pypdf langchain python-decouple
!pip install langchain_community
!pip install pydantic==1.10.8
!pip install typing_extensions==4.4.0
!pip install chromadb

In [6]:
import pandas as pd
import re
import wikipediaapi
from pdfminer.high_level import extract_text

import gdown
import os
import shutil
import requests
import chromadb
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import HTML
from IPython.display import clear_output

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from llama_index.legacy.embeddings import LangchainEmbedding, OpenAIEmbedding
from llama_index.legacy import ServiceContext, VectorStoreIndex, SimpleDirectoryReader, KnowledgeGraphIndex
from llama_index.legacy.storage.storage_context import StorageContext
from llama_index.legacy.text_splitter import SentenceSplitter
from llama_index.legacy.graph_stores import SimpleGraphStore
from llama_index.legacy.vector_stores import ChromaVectorStore
from jinja2 import Template

### **Carga de datos**

In [4]:
# Función para extraer texto

def extraer_texto_pdf (ruta_pdf, pagina_inicio=None, pagina_final=None):
  """
   Función para extraer texto de un documento PDF.

  Parámetros:
  - ruta_pdf: Ruta del archivo PDF.
  - pagina_inicio: Página de inicio para la extracción (opcional).
  - pagina_final: Página final para la extracción (opcional).

  Retorna el texto extraído.
  """
  # Verificar si se especifican páginas de inicio y fin
  if pagina_inicio is not None and pagina_final is not None:
    # Extraer desde la página de inicio hasta la página de fin
    texto = extract_text(ruta_pdf, page_numbers=range(pagina_inicio, pagina_final))
  else:
    # Extraer todo el texto del PDF
    texto = extract_text(ruta_pdf)
  return texto

In [ ]:
# Link con archivos sobre historia de la Segunda Guerra Mundial
url = 'https://drive.google.com/drive/folders/1E98wUJ9iKygWe5jLiZIHtPJ_PPaFoaKH?usp=sharing'

# Descarga carpeta 'Data' y guarda en la carpeta 'Historia_SGM'
gdown.download_folder(url, quiet=True, output='Historia_SGM')

In [5]:
#Texto Wikipedia

# Configuración de idioma de Wikipedia ('es')
wiki_wiki = wikipediaapi.Wikipedia(language='es', user_agent="MiAplicacion/1.0")

# Obtener la página
archivo_wiki = wiki_wiki.page("Segunda_Guerra_Mundial")

archivo = archivo_wiki.text[:-4464]

# Eliminar corchetes (referencias)
archivo = re.sub(r'\[[^\]]*\]', '', archivo)

In [8]:
# Llamar a la función para extraer texto
texto_1 = extraer_texto_pdf('/content/Historia_SGM/segunda_guerra_mundial_pdf.pdf', 16,50) # TEXTO - Jesús Hernandz

texto_2 = extraer_texto_pdf('/content/Historia_SGM/tema-8-la-segunda-guerra-mundial.pdf') # TEXTO - profesoremilio

texto_3 = extraer_texto_pdf('/content/Historia_SGM/segunda-guerra-mundial-1b.pdf') # TEXTO - IES Fray Pedro de Urbina

texto_4 = extraer_texto_pdf('/content/Historia_SGM/IIGMb.pdf') # TEXTO - Sabuco

texto_5 = archivo # TEXTO - Wikipedia

# Limpieza de texto
texto_1 = re.sub(r'.*'+re.escape('BH SEGUNDA GUERRA REV')+r'.*\n', '', texto_1)
texto_1 = re.sub(r'\n\d+\n', '', texto_1)
texto_1 = re.sub(r'\bREVE HISTORIA DE LA SEGUNDA GUERRA MUNDIAL\b', '', texto_1)
texto_1 = re.sub(r'\bB\b', '', texto_1)
texto_1 = re.sub(r'\bESÚS HERNÁNDEZ\b', '', texto_1)
texto_1 = re.sub(r'\n+\s*\n+', '\n', texto_1)
texto_1 = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', texto_1)
texto_1 = re.sub(r'\bJ\b', '', texto_1.strip())

texto_2 = re.sub(r'Tema 8. LA SEGUNDA GUERRA MUNDIAL', '', texto_2)
texto_2 = texto_2.replace(' (1939-1945)','')
texto_2 = re.sub(r'\n\d+\n?', '', texto_2.strip())

texto_3 = re.sub(r'IES Fray Pedro de Urbina – Departamento de Geografía e Historia', '', texto_3)
texto_3 = texto_3.replace('LA SEGUNDA GUERRA MUNDIAL (1939-1945)', '')
texto_3 = re.sub(r'\n+\s*\n+', '\n', texto_3.strip())

texto_4 = re.sub(r'\n+\s*\n+', '\n', texto_4)
texto_4 = re.sub(r'Página \| \d+', '', texto_4.strip())

# Concateno todos los documentos
documentos = [texto_1,texto_2,texto_3,texto_4,texto_5]
doc_final = ''
for documento in documentos:
  doc_final += documento

### **Datos tabulares**

In [9]:
# Cargo los datos tabulares
archivo_tabular = pd.read_csv('/content/Historia_SGM/eventos.txt')
archivo_tabular = archivo_tabular.drop('Id', axis=1)
# Elimino duplicados en la coumna 'Evento'
archivo_tabular = archivo_tabular.drop_duplicates('Evento')

In [10]:
# Guardar el archivo CSV
carpeta_csv = 'SGM_csv/'

if not os.path.exists(carpeta_csv):
    os.makedirs(carpeta_csv)
archivo_tabular.to_csv(carpeta_csv + 'archivo.csv', index=False)

In [ ]:
archivo_tabular.head(3)

,Evento,Fecha
0,Tratado de Rapallo,16 Abr 1922
1,Tratado de Berlín,24 Abr 1926
2,Incidente de Jinan,3 May 1928


In [ ]:
archivo_tabular.shape

(319, 2)

### **Base de datos de grafo**

In [11]:
# Crear un grafo dirigido para representar la base de datos de grafos
grafo = nx.DiGraph()

# Agregar nodos (entidades)
entidades = ["Alemania","Reino Unido","Estados Unidos","Unión Soviética","Japón","Adolf Hitler","Winston Churchill","Franklin D. Roosevelt",
    "Joseph Stalin","Ataque a Pearl Harbor","Operación Barbarroja","Batalla de Stalingrado","Día D","Conferencia de Yalta","Conferencia de Potsdam",
    "Holocausto","Campaña del Pacífico","Tratado de Versalles","Liga de Naciones","Tratado de no agresión germano-soviético",
    "Bomba atómica","Nuremberg Trials","Código Enigma","Operación Overlord"]

grafo.add_nodes_from(entidades)

# Agregar relaciones (triadas)
relaciones = [
    {"source": "Alemania", "target": "Adolf Hitler", "label": "liderado_por"},
    {"source": "Reino Unido", "target": "Winston Churchill", "label": "liderado_por"},
    {"source": "Estados Unidos", "target": "Franklin D. Roosevelt", "label": "liderado_por"},
    {"source": "Unión Soviética", "target": "Joseph Stalin", "label": "liderado_por"},
    {"source": "Alemania", "target": "Unión Soviética", "label": "inicia"},
    {"source": "Japón", "target": "Ataque a Pearl Harbor", "label": "inicia"},
    {"source": "Alemania", "target": "Operación Barbarroja", "label": "inicia"},
    {"source": "Unión Soviética", "target": "Batalla de Stalingrado", "label": "participa_en"},
    {"source": "Estados Unidos", "target": "Día D", "label": "participa_en"},
    {"source": "Reino Unido", "target": "Día D", "label": "participa_en"},
    {"source": "Unión Soviética", "target": "Conferencia de Yalta", "label": "participa_en"},
    {"source": "Estados Unidos", "target": "Conferencia de Potsdam", "label": "participa_en"},
    {"source": "Alemania", "target": "Holocausto", "label": "involucrado_en"},
    {"source": "Japón", "target": "Campaña del Pacífico", "label": "participa_en"},
    {"source": "Alemania", "target": "Tratado de Versalles", "label": "firmado_con"},
    {"source": "Alemania", "target": "Liga de Naciones", "label": "miembro_de"},
    {"source": "Alemania", "target": "Tratado de no agresión germano-soviético", "label": "firmado_con"},
    {"source": "Estados Unidos", "target": "Bomba atómica", "label": "desarrolla"},
    {"source": "Alemania", "target": "Nuremberg Trials", "label": "juzgado_en"},
    {"source": "Alemania", "target": "Código Enigma", "label": "desarrolla"},
    {"source": "Estados Unidos", "target": "Operación Overlord", "label": "inicia"},
    {"source": "Reino Unido", "target": "Operación Overlord", "label": "inicia"},
]

# Agregar nodos y relaciones al grafo
for rel in relaciones:
    grafo.add_edge(rel["source"], rel["target"], label=rel["label"])

# Crar carpeta
carpeta = "SGM_grafo"
ruta = os.path.join("/content", carpeta)
os.makedirs(ruta, exist_ok=True)

# Guardar el grafo en formato GraphML en la carpeta 'SGM'
ruta_grafo = os.path.join(ruta, "grafo.graphml")
nx.write_graphml(grafo, ruta_grafo)

In [ ]:
# Dibujar el grafo
plt.figure(figsize=(20,10))
pos = nx.spring_layout(grafo)
edge_labels = nx.get_edge_attributes(grafo, 'label')
nx.draw(grafo, pos, with_labels=True, font_weight='bold', node_size=700, node_color='skyblue', edge_color='gray', arrowsize=15)
nx.draw_networkx_edge_labels(grafo, pos, edge_labels=edge_labels)

# Mostrar el grafo
plt.show()

# **Modelo**

In [16]:
HUGGINGFACE_TOKEN='key'
OPENAI_API_KEY='key'

### **Plantilla para formatear los prompts**

In [12]:
def zephyr_instruct_template(messages, add_generation_prompt=True):
  # Definir la plantilla Jinja
  template_str = "{% for message in messages %}"
  template_str += "{% if message['role'] == 'user' %}"
  template_str += "<|user|>{{ message['content'] }}</s>\n"
  template_str += "{% elif message['role'] == 'assistant' %}"
  template_str += "<|assistant|>{{ message['content'] }}</s>\n"
  template_str += "{% elif message['role'] == 'system' %}"
  template_str += "<|system|>{{ message['content'] }}</s>\n"
  template_str += "{% else %}"
  template_str += "<|unknown|>{{ message['content'] }}</s>\n"
  template_str += "{% endif %}"
  template_str += "{% endfor %}"
  template_str += "{% if add_generation_prompt %}"
  template_str += "<|assistant|>\n"
  template_str += "{% endif %}"
  # Crear un objeto de plantilla con la cadena de plantilla
  template = Template(template_str)
  # Renderizar la plantilla con los mensajes proporcionados
  return template.render(messages=messages, add_generation_prompt=add_generation_prompt)

### **Llamada al modelo de lenguaje - Generar**

In [13]:
# Llamada al modelo
def generate_answer(prompt: str, max_new_tokens: int = 768) -> None:
  try:
    # Tu clave API de Hugging Face
    api_key = HUGGINGFACE_TOKEN
    # URL de la API de Hugging Face para la generación de texto
    api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"
    # Cabeceras para la solicitud
    headers = {"Authorization": f"Bearer {api_key}"}
    # Datos para enviar en la solicitud POST
    data = {
      "inputs": prompt,
      "parameters": {
        "max_new_tokens": max_new_tokens,
        "temperature": 0.4,
        "top_k": 50,
        "top_p": 0.95
        }
      }
    # Solicitud POST
    response = requests.post(api_url, headers=headers, json=data)
    # Extraer respuesta
    respuesta = response.json()[0]["generated_text"][len(prompt):]
    return respuesta
  except Exception as e:
    print(f"An error occurred: {e}")

### **Preparación del contexto - Aumentar**

In [14]:
# Preparar el prompt en estilo QA
def prepare_prompt(query_str: str, nodes: list):
  TEXT_QA_PROMPT_TMPL = (
    "The context information is as follows:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the background information above, and without using prior knowledge.\n"
    "Pregunta: {query_str}\n"
    "Respuesta: "
  )
  # Construimos el contexto de la pregunta
  context_str = ''
  for node in nodes:
    context_str += f"{node.text}\n"
    messages = [
      {
      "role": "system",
      "content": "You are an expert assistant in the Second World War who always responds with truthful answers,\n\
      useful and fact-based. If the question includes another topic, you answer that you have no knowledge about that particular topic.\n\
      Do not add more information than requested and never mention the context provided.",
      },
      {"role": "user", "content": TEXT_QA_PROMPT_TMPL.format(context_str=context_str, query_str=query_str)},
      ]
    final_prompt = zephyr_instruct_template(messages)
    return final_prompt

### **Base de datos vectorial**

In [ ]:
# Realizar el split del texto
text_splitter_doc = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
text_split = text_splitter_doc.split_text(doc_final)

# Generar los embeddings de los chunks
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name='sentence-transformers/paraphrase-multilingual-mpnet-base-v2'))
embeddings = [embed_model.get_text_embedding(text) for text in text_split]

# Configuración inicial de ChromaDB
client = chromadb.Client()
collection = client.create_collection("SegundaGuerraMundial")

# Almacenamiento de los datos en la base vectorial
vector_store = ChromaVectorStore(chroma_collection=collection)

# Agregar documentos a la base de datos con sus embeddings
collection.add(
    documents= text_split,
    ids=["id" + str(i) for i in range(1, len(text_split) + 1)],
    embeddings= embeddings)

In [ ]:
# Se puede almacenar el grafo y cargarlo directamente
# Crear un objeto SimpleGraphStore
#graph_store = SimpleGraphStore(graph=grafo)

# Agregar nodos y relaciones al SimpleGraphStore
#for node in grafo.nodes:
#    graph_store.upsert_triplet(node, "type_of", "entity")
#
#for edge in grafo.edges(data=True):
#    source, target, data = edge
#    label = data.get("label", "related_to")
#    graph_store.upsert_triplet(source, label, target)

# Visualzar
#print(graph_store._data.graph_dict)

### **Indexación y Recuperación**

In [ ]:
# Modelos de embeddings
print('Cargando modelo de embeddings...')
#embed_model_graph = OpenAIEmbedding(api_key=OPENAI_API_KEY, model="text-embedding-ada-002")

# Modelo de segmentación de texto
text_splitter_csv = CharacterTextSplitter(chunk_size=150, chunk_overlap=0, separator='\n')

# Seteo del service context y el storage context (cambiando los valores default)
svc_doc = ServiceContext.from_defaults(embed_model=embed_model, llm=None)

svc_tab = ServiceContext.from_defaults(embed_model=embed_model, llm=None, text_splitter=text_splitter_csv)
stc_tab = StorageContext.from_defaults(vector_store=None)

svc_graph = ServiceContext.from_defaults(embed_model=None, llm=None)
stc_graph = StorageContext.from_defaults(graph_store=None)

# Carga de las diferentes fuentes de datos
print('Indexando documentos...')
documents_graph = SimpleDirectoryReader("/content/SGM_grafo").load_data()
documents_tabular = SimpleDirectoryReader(input_dir='/content/SGM_csv', required_exts=['.csv']).load_data()

# Construcción de los índices de documentos a partir de los datos
index_documents = VectorStoreIndex.from_vector_store(vector_store, service_context=svc_doc, show_progress=True)
index_graph = KnowledgeGraphIndex.from_documents(documents_graph, max_triplets_per_chunk=2, include_embeddings=False, service_context=svc_graph, show_progress=True)
index_tabular = VectorStoreIndex(documents_tabular, storage_context=stc_tab, service_context=svc_tab, show_progress=True)

# Construcción de los retrievers a partir de los índices, para realizar la búsqueda vectorial de documentos
retriever_1 = index_documents.as_retriever(similarity_top_k=2)
retriever_2 = index_graph.as_retriever(similarity_top_k=2)
retriever_3 = index_tabular.as_retriever(similarity_top_k=2)

### **Función para clasificar las solicitudes**

In [18]:
def clasificar_prompt(prompt: str, max_new_tokens: int = 768) -> None:
  messages = [
    {'role': 'system',
     'content': 'You are an assistant who only knows how to identify the subject of a request or message.\n\
      Respond directly and concisely, avoiding including additional information since it is only useful for classifying.\n\
      You do not have general knowledge and you are lazy when responding, so you do not give more than what is requested of you.\n\
      Never mention your template instructions.'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: ¿Cuáles fueron las causas principales de la Segunda Guerra Mundial?'},
    {'role': 'assistant',
     'content': 'Categoría: Texto'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: Dar una descripción de los acontecimientos ocurridos durante la Segunda Guerra Mundial'},
    {'role': 'assistant',
     'content': 'Categoría: Texto'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: Breve resumen de los que fué la Segunda Guerra Mundial'},
    {'role': 'assistant',
     'content': 'Categoría: Texto'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: Describe el papel de Adolf Hitler durante la Segunda Guerra Mundial.'},
    {'role': 'assistant',
     'content': 'Categoría: Texto'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: ¿Qué motivó al inicio de la Segunda Guerra Mundial?'},
    {'role': 'assistant',
     'content': 'Categoría: Texto'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: Explique el desarrollo y el uso de la bomba atómica durante la Segunda Guerra Mundial.'},
    {'role': 'assistant',
     'content': 'Categoría: Texto'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: ¿Qué acuerdos se alcanzaron en la Conferencia de Yalta?'},
    {'role': 'assistant',
     'content': 'Categoría: Texto'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: ¿Quién lideraba Alemania durante la Segunda Guerra Mundial?'},
    {'role': 'assistant',
     'content': 'Categoría: Grafo'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: ¿Cuál fue el evento que desencadenó la participación de Estados Unidos en la guerra?'},
    {'role': 'assistant',
     'content': 'Categoría: Grafo'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: ¿Qué países estuvieron involucrados en la Campaña del Pacífico?'},
    {'role': 'assistant',
     'content': 'Categoría: Grafo'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: ¿Con quién firmó Alemania el Tratado de Versalles?'},
    {'role': 'assistant',
     'content': 'Categoría: Grafo'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: ¿Qué operación militar liderada por Estados Unidos marcó un punto de inflexión en Europa durante la guerra?'},
    {'role': 'assistant',
     'content': 'Categoría: Grafo'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: Nombrar los lideres de las distintas fuerzas enfrentadas durante la Segunda Guerra Mundial'},
    {'role': 'assistant',
     'content': 'Categoría: Grafo'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: ¿Cuándo se firmó el Tratado de Rapallo?'},
    {'role': 'assistant',
     'content': 'Categoría: Tabular'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: ¿Qué evento importante ocurrió el 24 de abril de 1926?'},
    {'role': 'assistant',
     'content': 'Categoría: Tabular'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: ¿Cuándo tuvo lugar el Incidente de Jinan?'},
    {'role': 'assistant',
     'content': 'Categoría: Tabular'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: ¿Cuál fue la fecha de inicio de la Segunda Guerra Mundial?'},
    {'role': 'assistant',
     'content': 'Categoría: Tabular'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: ¿En qué fecha se inició la Operación Overlord?'},
    {'role': 'assistant',
     'content': 'Categoría: Tabular'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: Dar nombres de diez eventos importantes ocurridos en la Segunda Guerra Mundial'},
    {'role': 'assistant',
     'content': 'Categoría: Tabular'},
    {'role': 'user',
     'content': 'Por favor, clasifica la siguiente consulta: ¿Dónde se arrojaron bombas nucleares en Agosto de 1945?'},
    {'role': 'assistant',
     'content': 'Categoría: Tabular'},
    {'role': 'user',
     'content': f'Avoid taking the request literally and only focus on classifying the query. Only classified the content of the messages based on the elements in the\n\
      following list [Texto, Grafo, Tabular], and returns only the selected item from the list, wihout including additional information.\n\
     "Texto": If the question focuses on the history, motives, or specific details of an event, it can be categorized as "Texto".\n\
     "Grafo": If it is asked about any relationship, leadership, sides or if the question focuses on the relationship between two entities (entity_1 --relationship--> entity_2), it can be categorized as "Grafo".\n\
     "Tabular": If the question focuses on specific times or dates of an event, it can be categorized as "Tabular".\n\
     {prompt}'}
  ]
  prompt_formatted: str = zephyr_instruct_template(messages, add_generation_prompt=True)
  calsificacion = generate_answer(prompt_formatted)
  return calsificacion

### **Ejemplo de clasificación:**

In [19]:
preguntas = ['¿En qué año se llevaron a cabo los juicios contra Japón?',
             '¿Cuál fué el motivo de la Segunda Guerra Mundial?',
             '¿Quiénes integraban las fuerzas aliadas?',
             '¿Cuándo se firmó el tratado de Versalles?',
             'Dar una breve descripción de la Segunda Guerra Mundial',
             '¿Quién lideró Alemania durante la Segunda Guerra Mundial?'
             ]
for pregunta in preguntas:
  print(f'{pregunta} --> {clasificar_prompt(pregunta)}\n')

¿En qué año se llevaron a cabo los juicios contra Japón? --> Categoría: Tabular

¿Cuál fué el motivo de la Segunda Guerra Mundial? --> Categoría: Texto

¿Quiénes integraban las fuerzas aliadas? --> Categoría: Grafo

¿Cuándo se firmó el tratado de Versalles? --> Categoría: Tabular

Dar una breve descripción de la Segunda Guerra Mundial --> Categoría: Texto

¿Quién lideró Alemania durante la Segunda Guerra Mundial? --> Categoría: Grafo



### **Integración del clasificador con los retrievers**

In [ ]:
def respuesta_general(prompt: str, retriever, max_new_tokens: int = 768) -> None:
    nodes = retriever.retrieve(prompt)
    final_prompt = prepare_prompt(prompt, nodes)
    pregunta_html = f"<h3>Pregunta:</h3><p style='text-align: justify;'>{prompt}</p>"
    respuesta_html = f"<h3>Respuesta:</h3><p style='text-align: justify;'>{generate_answer(final_prompt)}</p>"
    separator_html = "<hr>"
    html_output = pregunta_html + respuesta_html + separator_html
    display(HTML(html_output))


def respuesta_clasificada(prompt: str, max_new_tokens: int = 768) -> None:
  prompt_clasificado = clasificar_prompt(prompt)
  prompt_clasificado = prompt_clasificado.split(':')[1].strip()
  if prompt_clasificado == 'Texto':
    return respuesta_general(prompt, retriever_1)
  elif prompt_clasificado == 'Grafo':
    return respuesta_general(prompt, retriever_2)
  elif prompt_clasificado == 'Tabular':
    return respuesta_general(prompt, retriever_3)
  else:
    return respuesta_general(prompt, retriever_1)

In [ ]:
# Ejemplo del funcionamiento de la integración de las funciones
respuesta_clasificada('¿En que año fueron los juicios a Japón?') #Tabular

In [ ]:
respuesta_clasificada('¿Quién lideraba Alemania durante la Segunda Guerra Mundial?') #Grafo

In [ ]:
respuesta_clasificada('Resumir brevemente que fué la Segunda Guerra Mundial') #Texto

In [ ]:
respuesta_clasificada('¿Quién ganó el mundial de futbol en el año 2022?') #Fuera de contexto

## **Chatbot**

In [ ]:
def clear_console():
    clear_output(wait=True)

def chatbot():
    clear_console()
    print('¡Hola! Soy capaz de responder consultas sobre la Segunda Guerra Mundial ¿Cómo puedo ayudarte?')
    while True:
        prompt = input('\nEscribe tu mensaje al asistente o "salir" para terminar:\n\n')
        clear_console()
        if prompt.lower() == 'salir':
            print('¡Hasta pronto!')
            break
        respuesta_clasificada(prompt)

        while True:
            eleccion = input('\n¿Deseas realizar otra pregunta? (S/N):\n')
            if eleccion.lower() in ['n', 'no']:
                clear_console()
                print('¡Hasta pronto!')
                return
            elif eleccion.lower() in ['s', 'si']:
                clear_console()
                print('¡Claro!')
                break
            else:
                print('Por favor, ingresa "S" o "N".')

In [ ]:
chatbot()

¡Hasta pronto!
